Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [1]:
NAME = "Adam Bakopolus"
COLLABORATORS = ""

# Presenting Uncertainty
## School of Information, University of Michigan

## Week 3: Assignment Overview
Version 1.2
### The objectives for this week are for you to:
- learn how to construct hypothetical outcome plots (HOPs) and spaghetti plots for a fit line
- practice making HOPs and spaghetti plots on Boston Housing Prices dataset

In [2]:
import time
import altair as alt
import pandas as pd
import ipywidgets as widgets
from ipywidgets import interact
from sklearn import linear_model
from sklearn import gaussian_process
import numpy as np

import operator
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures




# Part 1: Learn to plot HOPs and spaghetti plots for linear regression (12 points)

The following salary dataset describes the relationship between someone's salary and the number of years of experience someone has. In this section, we will construct an animated hypothetical outcome plot (HOP) and a spaghetti plot of a linear regression fit to this dataset.

In [3]:
#load dataset
salary_df = pd.read_csv("asset/Salary_Data.csv")
salary_df.head()

,YearsExperience,Salary
0,1.1,39343.0
1,1.3,46205.0
2,1.5,37731.0
3,2.0,43525.0
4,2.2,39891.0


## 1.1 Construct the basic building blocks of a HOPs visualization

In order to construct a HOPs visualization, we need the following functions:

1. A function to construct an Altair chart of the data: `get_salary_points_chart()`
2. A function to get one bootstrap sample of the linear regression fit: `get_one_bootstrap_salary_fit()`
3. A function to construct an Altair chart of one linear regression fit line: `get_salary_linear_fit_chart()`

Then we will combine all these functions together to make an animation.

### Question 1.1.1 Plot the data (5 points)

Construct a function, `get_salary_points_chart()`, which plots the data in `salary_df` as a scatterplot. The output should look like this:

![A scatterplot of Years of Experience (x axis) against Salary (y axis)](asset/assignment3_salary_points_chart.png)

In [4]:
def get_salary_points_chart():
    '''
    This function should return an altair plot object that is a scatterplot of
    the salary data, with YearsExperience on the x axis and Salary on the y axis
    '''
    salary = alt.Chart(salary_df).mark_circle(color="black") \
                .encode(
                alt.X("YearsExperience"), alt.Y("Salary")
                       )

    return salary
    #raise NotImplementedError()
    
get_salary_points_chart()

alt.Chart(...)

### Question 1.1.2 Bootstrap one linear regression fit (2 points)

We will need a function that returns one bootstrap sample of the regression fit. That is, it resamples the dataset with replacement, then fits a linear regression to the data. Fill in the code below to complete the function:

In [5]:
def get_one_bootstrap_salary_fit():
    '''
    Returns a sklearn.linear_model.LinearRegression model representing 
    a fit to a bootstrap-resampled version of salary_df
    '''
    
    #resample the data with replacement (replace=True) to a data frame with 
    #the same number of data points (frac=1.0)
    resampled_df = salary_df.sample(frac=1.0, replace=True)

    #fit model to resampled data
    X = resampled_df[['YearsExperience']] #[[ ]] subsets so X remains a DataFrame
    y = resampled_df['Salary']            #y should be an array, so we use [ ]
    
    # insert code below using LinearRegression to return a linear regression model
    # with predictor X and outcome variable y
    return linear_model.LinearRegression().fit(X, y)
    #raise NotImplementedError()

In [6]:
np.random.seed(1234)
fit = get_one_bootstrap_salary_fit()
assert np.abs(fit.coef_[0] - 10004) < 0.5, "Bootstrap linear regression: slope coefficient does not match the expected value"
assert np.abs(fit.intercept_ - 21485) < 0.5, "Bootstrap linear regression: intercept does not match the expected value"

We can use this function to get a single sample from the bootstrap sampling distribution of the fit (e.g., its slope and intercept). Each time you run the following cell you should get slightly different values:

In [7]:
salary_reg = get_one_bootstrap_salary_fit()
print("Bootstrapped intercept: ", salary_reg.intercept_)
print("Bootstrapped slope:     ", salary_reg.coef_[0])

Bootstrapped intercept:  23608.862570184552
Bootstrapped slope:      9759.092501075675


### Question 1.1.3 Construct an Altair chart of one regression fit (5 points)

To construct a chart of a fit line or fit curve, we first need a *prediction grid*: a set of x values we want to use to make predictions. This should be in the same form as the input to the regression function (i.e., a DataFrame). 

For this example, we will use evenly-spaced values of `"YearsExperience"`, the x value in our charts. Because it is a linear fit, we strictly speaking only need 2 values, but we will use more (101) because it generalizes better. When you plot non-linear relationships (as we will in Part 2), you need a large number of points in your prediction grid so that the curve is smooth.

In [8]:
# construct a prediction grid for the salary dataset with 101 
# evenly-spaced values from the minimum to maximum number of years of experience
salary_pred_grid = pd.DataFrame({'YearsExperience': np.linspace(
    salary_df['YearsExperience'].min(), 
    salary_df['YearsExperience'].max(), 
    num=101
)})

Complete the `get_salary_linear_fit_chart()` so that it displays a single fit line from the linear regression fit passed in to it. The chart should look like this:

![A line chart of Years of Experience (x axis) against Salary (y axis)](asset/assignment3_salary_line_chart.png)

In [9]:
def get_salary_linear_fit_chart(salary_reg, opacity=0.5):
    '''
    Takes a single linear regression fit (as returned by `get_one_bootstrap_salary_fit()`) and
    returns an Altair chart plotting the fit line
    
    Parameters:
    
    - salary_reg: A regression fit
    - opacity: The opacity of the output line
    '''
    #use the model to predict the mean Salary at each x position
    pred_df = pd.DataFrame({
        'YearsExperience': salary_pred_grid['YearsExperience'],
        'Salary': salary_reg.predict(salary_pred_grid)
    })

    #insert code to return an Altair chart showing the fit line using `pred_df`
    #remember to set the opacity of the line mark to the `opacity` value
    #passed into this function (e.g. `mark_line(opacity=opacity)`)
    fit_line = alt.Chart(pred_df).mark_line(color="red", opacity = opacity) \
                  .encode(
                     alt.X("YearsExperience"), alt.Y("Salary")
                       )
    
    return fit_line
    #raise NotImplementedError()

get_salary_linear_fit_chart(salary_reg)

alt.Chart(...)

## 1.2 Construct HOPs of the salary data

Now that you have all the pieces, you should be able to put them together to construct a HOPs visualization.

First, run the following code chunk a few times: you should notice that the fit line moves each time you run it.

In [10]:
points_chart = get_salary_points_chart()
salary_reg = get_one_bootstrap_salary_fit()
line_chart = get_salary_linear_fit_chart(salary_reg)
line_chart + points_chart

alt.LayerChart(...)

We will use the `interact()` function to run the above code to generate each frame needed in our HOPs. Run the following code, then press the Play button to start the animation:

In [11]:
def get_one_frame(i):
    '''
    Return one frame in the animation
    '''

    time.sleep(.2)

    # get the point chart
    points_chart = get_salary_points_chart()
    
    # fit one bootstrap regression
    salary_reg = get_one_bootstrap_salary_fit()
    
    # get the line chart
    line_chart = get_salary_linear_fit_chart(salary_reg)
    
    #return the combined points + lines chart
    return line_chart + points_chart

interact(get_one_frame, i = widgets.Play(
    value=0,
    min=0,
    max=100,
    step=1,
    description="Press play",
    disabled=False))

interactive(children=(Play(value=0, description='Press play'), Output()), _dom_classes=('widget-interact',))

<function __main__.get_one_frame(i)>

## 1.3 Construct a spaghetti plot of the salary data

The same functions we used to make the HOPs chart above can be used to make a spaghetti plot as well. This time, we will combine all the line charts together instead of playing them frame-by-frame. First, we make a list containing all the line charts (in the `line_charts` variable), then we use `alt.layer()` to layer all of the line charts together. Finally, we add on the chart of the points:

In [12]:
B = 50

# get `B` bootstrapped fit line charts
# Note opacity=0.1 sets the line opacity so it is easier to see the overlapping lines. Make
# sure your get_salary_linear_fit_chart() function (defined above) properly uses the opacity argument!
line_charts = [get_salary_linear_fit_chart(get_one_bootstrap_salary_fit(), opacity=0.1) for _ in range(B)]

#combine all the line charts together and layer on the points chart
alt.layer(*line_charts) + get_salary_points_chart()

alt.LayerChart(...)

# Part 2: Spaghetti plots for Polynomial Regression (5 points)

To demonstrate the difference in how you must modify your code to fit a new model, in this section we show how to create spaghetti plots for a polynomial regression. We will follow the same steps as before:

1. A function to construct an Altair chart of the data: `get_poly_points_chart()`
2. A function to get one bootstrap sample of the linear regression fit: `get_one_bootstrap_poly_fit()`
3. A function to construct an Altair chart of one linear regression fit line: `get_poly_fit_chart()`

We will generate a dataset with two variables (`x` and `y`), then draw spaghetti plots of a polynomial fit to the dataset.

## 2.1 Generate dataset

First, generate the dataset:

In [13]:
#prepare dataset
np.random.seed(42)
n = 25

original_x = 5 - 4 * np.random.normal(0, 1, n)
original_y = -2 + 3*original_x - 5*(original_x ** 2) + 7*(original_x ** 3) + np.random.normal(0, 1000, n)

poly_df = pd.DataFrame({'x': original_x, 'y': original_y})

## 2.2 Define helper functions

We'll define the polynomial points chart and draw it:

In [14]:
def get_poly_points_chart():
    '''
    This function should return an altair plot object that is a scatterplot of
    the salary data, with YearsExperience on the x axis and Salary on the y axis
    '''
    return alt.Chart(poly_df).mark_circle(color="black").encode(
        x='x',
        y='y'
    )

get_poly_points_chart()

alt.Chart(...)

Then we define the `get_one_bootstrap_poly_fit()` and `get_poly_fit_chart()` functions so we can draw a single fit:

In [15]:
#prediction grid
poly_pred_grid = pd.DataFrame({
    "x": np.linspace(poly_df['x'].min(), poly_df['x'].max(), num=101)
})

def get_one_bootstrap_poly_fit():
    '''Get one bootstrap sampled polynomial regression fit to the data'''
    #resample the data with replacement (replace=True) to a data frame with 
    #the same number of data points (frac=1.0)
    resampled_df = poly_df.sample(frac=1.0, replace=True)

    #fit model to resampled data
    X = resampled_df[['x']] #[[ ]] subsets so X remains a DataFrame
    y = resampled_df['y']   #y should be an array, so we use [ ]
    
    #x must be transformed into polynomials (e.g. x, x^2, x^3 ... up to the value of `degree`)
    polynomial_features = PolynomialFeatures(degree=2)
    X_poly = polynomial_features.fit_transform(X)
    poly_reg = linear_model.LinearRegression()
    poly_reg.fit(X_poly, y)
    
    return poly_reg

def get_poly_fit_chart(poly_reg, opacity=0.5):
    '''
    Takes a single polynomial regression fit (as returned by `get_one_bootstrap_poly_fit()`) and
    returns an Altair chart plotting the fit curve
    
    Parameters:
    
    - poly_reg: A regression fit
    - opacity: The opacity of the output line
    '''
    #use the model to predict y at each x position
    polynomial_features = PolynomialFeatures(degree=2)
    pred_df = pd.DataFrame({
        'x': poly_pred_grid['x'],
        'y': poly_reg.predict(polynomial_features.fit_transform(poly_pred_grid))
    })

    #return an Altair chart showing the fit line
    return alt.Chart(pred_df).mark_line(
        opacity=opacity,
        color='red'
    ).encode(
        x='x',
        y='y'
    )

poly_reg = get_one_bootstrap_poly_fit()
get_poly_fit_chart(poly_reg)

alt.Chart(...)

# 2.3 Draw spaghetti plot for polynomial regression

### Question 2.3.1 Draw a spaghetti plot for the above polynomial regression (5 points)

Using the helper functions defined above (`get_poly_points_chart()`, `get_one_bootstrap_poly_fit()`, and `get_poly_fit_chart()`), draw a spaghetti plot for the example polynomial regression data. Your output should look something like this:

![Polynomial spaghetti plot fit](asset/assignment3_poly.png)



In [16]:
B = 50

# get `B` bootstrapped fit line charts
# Note opacity=0.1 sets the line opacity so it is easier to see the overlapping lines.
poly_charts = [get_poly_fit_chart(get_one_bootstrap_poly_fit(), opacity=0.1) for _ in range(B)]

#combine all the line charts together and layer on the points chart
alt.layer(*poly_charts) + get_poly_points_chart()
#raise NotImplementedError()

alt.LayerChart(...)

# Part 3: Diabetes dataset (23 points)

In Part 3, we switch to a new data set that describes diabetes disease progression (a metric that captures the progression of diabetes, where higher scores represent a more advanced case of diabetes) and multiple predictors. Apply what you have learned above to construct hypothetical outcome plots and spaghetti plots for the relationship between diabetes disease progression (`disease_progression`) and the `hdl` variable (high-density lipoproteins -- "good chloestrol" that transports chloestrol to the liver). A visualization of the relationship can be found below:

In [17]:
from sklearn.datasets import load_diabetes
X, y = load_diabetes(return_X_y=True)

#create a dataframe containing predictors (housing_X) and the response variable (housing_y)

diabetes_X = pd.DataFrame(X, columns=["age","sex","bmi","bp", "tc", "ldl", "hdl","tch", "ltg", "glu"])
diabetes_y = pd.DataFrame(y, columns=["disease_progression"])

#also create a combined data frame with both predictors and response variables
diabetes_df = pd.concat([diabetes_y, diabetes_X], axis=1)

#show the hdl versus disease_progression
alt.Chart(diabetes_df).mark_point().encode(
    x="hdl",
    y="disease_progression"
)

alt.Chart(...)

## 3.1 HOPs and spaghetti plots

Use HOPs and spaghetti plots to visualize a regression model predicting `disease_progression` using `hdl`. You can use any model type you like, including linear regression, polynomial regression, or any other regression model type.

### Question 3.1.1 Define helper functions (10 points)

Define the helper functions you will need, including:

1. A function to construct an Altair chart of the data: `get_diabetes_points_chart()`
2. A function to get one bootstrap sample of the fit: `get_one_bootstrap_diabetes_fit()`
3. A function to construct an Altair chart of one regression fit curve: `get_diabetes_fit_chart()`


In [18]:
# define your helper functions below. Hint: this is also a good place to define a prediction grid

def get_diabetes_points_chart():

    return alt.Chart(diabetes_df).mark_circle(color="black").encode(
        x='hdl',
        y='disease_progression'
    )

#prediction grid
diabetes_pred_grid = pd.DataFrame({
    "hdl": np.linspace(diabetes_df['hdl'].min(), diabetes_df['hdl'].max(), num=101)
})

def get_one_bootstrap_diabetes_fit():
    '''Get one bootstrap sampled polynomial regression fit to the data'''
    #resample the data with replacement (replace=True) to a data frame with 
    #the same number of data points (frac=1.0)
    resampled_df = diabetes_df.sample(frac=1.0, replace=True)

    #fit model to resampled data
    X = resampled_df[['hdl']] #[[ ]] subsets so X remains a DataFrame
    y = resampled_df['disease_progression']   #y should be an array, so we use [ ]
    
    #x must be transformed into polynomials (e.g. x, x^2, x^3 ... up to the value of `degree`)
    polynomial_features = PolynomialFeatures(degree=2)
    X_poly = polynomial_features.fit_transform(X)
    diabetes_reg = linear_model.LinearRegression()
    diabetes_reg.fit(X_poly, y)
    
    return diabetes_reg

def get_diabetes_fit_chart(diabetes_reg, opacity=0.5):

    #use the model to predict y at each x position
    polynomial_features = PolynomialFeatures(degree=2)
    pred_df = pd.DataFrame({
        'x': diabetes_pred_grid['hdl'],
        'y': diabetes_reg.predict(polynomial_features.fit_transform(diabetes_pred_grid))
    })

    #return an Altair chart showing the fit line
    return alt.Chart(pred_df).mark_line(
        opacity=opacity,
        color='red'
    ).encode(
        x='x',
        y='y'
    )

### Question 3.1.2 Create a spaghetti plot for your model (5 points)

Using the helper functions you created above, visualize a spaghetti plot of your model below.


In [19]:
B = 50

# get `B` bootstrapped fit line charts
# Note opacity=0.1 sets the line opacity so it is easier to see the overlapping lines.
diabetes_charts = [get_diabetes_fit_chart(get_one_bootstrap_diabetes_fit(), opacity=0.1) for _ in range(B)]

#combine all the line charts together and layer on the points chart
alt.layer(*diabetes_charts) + get_diabetes_points_chart()
#raise NotImplementedError()

alt.LayerChart(...)

### Question 3.1.3 Create a HOPs chart for your model (5 points)

Using the helper functions you created above, visualize a HOPs chart of your model below.

In [20]:
def get_one_frame(i):
    '''
    Return one frame in the animation
    '''

    time.sleep(.2)

    # get the point chart
    points_chart = get_diabetes_points_chart()
    
    # fit one bootstrap regression
    diabetes_reg = get_one_bootstrap_diabetes_fit()
    
    # get the line chart
    line_chart = get_diabetes_fit_chart(diabetes_reg)
    
    #return the combined points + lines chart
    return line_chart + points_chart

interact(get_one_frame, i = widgets.Play(
    value=0,
    min=0,
    max=100,
    step=1,
    description="Press play",
    disabled=False))
#raise NotImplementedError()

interactive(children=(Play(value=0, description='Press play'), Output()), _dom_classes=('widget-interact',))

<function __main__.get_one_frame(i)>

### Question 3.1.4 Reflect on your model (3 points)

Given the visualizations above, reflect on the model you chose and the uncertainty in the relationship between `hdl` and `disease_progression` in these data. Discuss both small world and large world uncertainty. 

The model above is a polynomial (degree = 2) regression that shows a slight negative correlation between HDL and diabetes disease progression. As HDL increases, the expectation is that disease progression will decrease and vice versa. As can be seen, the relationship between HDL and disease progression when looking just at the points is dubious. Both the linear and polynomial regressions resulted in a relatively poor fit and, in my mind, neither would be suitable when attempting to estimate disease progression based on HDL.

In regards to earlier lecture discussions around uncertainty specifically, there is both small world and large world uncertainty to be considered as part of this work. For small world uncertainty, this is an example of parameter uncertainty as there is uncertainty around the prediction of disease progression, a parameter of interest. While this uncertainty will decrease as the sample increases, there is a question around how strong an estimate this model may provide given the available information and the size of the data.

While these are small world uncertainty issues, there are also large world uncertainty issues to consider. One concern would be around model choice: is a polynomial regression the appropriate model to choose or should logarithmic, exponential, etc. models been considered to better capture the data? As noted earlier, both linear and polynomial regressions poorly fit to the data. Therefore, a model choice would add to the uncertainty around our approach. Additionally, the model assumptions themselves are important to consider for uncertainty. It is possible that a better model could have been generated if more factors than just HDL were considered, like the readily available age, sex, BMI, LDL, BP, etc. data. By not including these and just limiting to HDL, our model again may not be an accurate predictor of disease progression, again adding to uncertainty. Lastly, while the diabetes data was pulled directly from the scikit learn package, data collection is also a large world uncertainty factor to consider, and data should be carefully reviewed to ensure its completeness and accuracy. Therefore, this week's work has both small and large world uncertainty to consider.

Please remember to submit both the HTML and .ipynb formats of your completed notebook. When generating your HTML, be sure to run your complete code first before downloading as HTML. Please remember to work on your explanations and interpretations!